In [1]:
import random
import numpy as np
import math
import time
import os

In [2]:
def euclidean_distance(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [3]:
def genetic_backpack(max_capacity, w, c):
    N = len(w)
    n_chromo = 200
    dna = np.zeros(N*n_chromo).reshape(n_chromo, N)
    k = 2
    # генерируем хромосомы
    for i in range(n_chromo):
        for j in range(N):
            dna[i][j] = random.randint(0,1)
    #print('Популяция:\n', dna)
    def remove_random_item(chromo):
        one_positions = []
        for i in range(len(chromo)):
            if chromo[i]:
                one_positions.append(i)
        chromo[one_positions[random.randint(0, len(one_positions) - 1)]] = 0
    
    best_solution = []
    for i in range(100):
        # Считаем вес и стоимость
        def fitness_function(chromo, w, c):
            temp_w, temp_p = 0, 0
            for j in range(N):
                if chromo[j]:
                    temp_w += w[j]
                    temp_p += c[j]
            return temp_p, temp_w
        total_chromo = []
        for i in range(n_chromo):
            temp_p, temp_w = fitness_function(dna[i], w, c)

            while temp_w > max_capacity:
                remove_random_item(dna[i])
                temp_p, temp_w = fitness_function(dna[i], w, c)
            total_chromo.append((i, temp_p, temp_w))
        #print('Популяция после подсчета фит. функции:\n', dna)  

        # Сортируем
        total_chromo = sorted(total_chromo, key=lambda x: x[1], reverse=True)
        #print(total_chromo)
        best_solution = [dna[total_chromo[0][0]], total_chromo[0][1], total_chromo[0][2]]
        
        # Находим распределение вероятностей
        fitness_sum = 0
        for x in total_chromo:
            fitness_sum += x[1]
        distribution = []
        for chromo in dna:
            distribution.append(fitness_function(chromo, w, c)[0] / fitness_sum)
        d = n_chromo // 10
        for i in range(n_chromo - d, n_chromo):
            distribution[i] *= 2
        #distribution.sort(reverse=True)
        #print(sorted(distribution), reverse=True)
        # выбираем для селекции
        chromos_selected = random.choices(dna, weights=distribution, k=k)
        # чтобы не брать одинаковые
        while len(np.unique(chromos_selected, axis=0)) < k:
            chromos_selected = random.choices(dna, weights=distribution, k=k)
        #print('Выбрали для селекции:\n',chromos_selected)
        
        indices = []
        for i in range(len(dna)):
            if (dna[i] == chromos_selected[0]).all() or (dna[i] == chromos_selected[1]).all():
                indices.append(i)
        dna = np.delete(dna, [indices[0], indices[1]], 0)

        # делаем кроссовер
        crossover_point = random.randint(1, N-1)
        tmp = chromos_selected[1][:crossover_point].copy()
        chromos_selected[1][:crossover_point], chromos_selected[0][:crossover_point] = chromos_selected[0][:crossover_point], tmp
        #print('После селекции:\n', chromos_selected)

        # мутация
        for j in range(len(chromos_selected)):
            for i in range(len(chromos_selected[j])):
                if random.randint(1, 50) == 1:
                    chromos_selected[j][i] = (chromos_selected[j][i] + 1) % 2
        #print('После мутации:\n', chromos_selected)

        # считаем ф-ф полученных хромо, выравниваем веса
        fitness_results_selected = []
        for i in range(len(chromos_selected)):
            temp_p, temp_w = fitness_function(chromos_selected[i], w, c)
            while temp_w > max_capacity:
                remove_random_item(chromos_selected[i])
                temp_p, temp_w = fitness_function(chromos_selected[i], w, c)
            fitness_results_selected.append((i, temp_p, temp_w))

        dna = np.append(dna, [chromos_selected[0]], 0)
        dna = np.append(dna, [chromos_selected[1]], 0)

        fitness_results_selected.sort(key=lambda x: x[1], reverse=True)

        if fitness_results_selected[0][1] > best_solution[2]: #total_chromo[0][1]:
            best_solution = [chromos_selected[fitness_results_selected[0][0]], fitness_results_selected[0][1], fitness_results_selected[0][2]]  #fitness_results_selected[0]] #(chromos_selected[fitness_results_selected[0][0]], fitness_results_selected[0][1], fitness_results_selected[0][2])
        #else:
            #best_solution = dna[total_chromo[0][0]], total_chromo[0][1], total_chromo[0][2]
    return best_solution

In [13]:
p_c = []
p_p = []
p_s = []
p_w = []
for i in range(1, 8):
    p_c.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_c.txt", "r"))
    p_p.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_p.txt", "r"))
    p_s.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_s.txt", "r"))
    p_w.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_w.txt", "r"))
for test_num in range(7):
    print(f"BENCHMARK p0{test_num+1}")
    w = int(p_c[test_num].read())
    items_c = []
    for line in p_p[test_num]:
        items_c.append(int(line))
        
    items_w = []
    for line in p_w[test_num]:
         items_w.append(int(line))

    items_true = []
    w_true = 0
    c_true = 0
    for line in p_s[test_num]:
        items_true.append(int(line))
    for j in range(len(items_true)):
        if items_true[j] == 1:         
            w_true += items_w[j]
            c_true += items_c[j]
                
    answers = []
    timeExecuted = 0
    for i in range(100):
        start = time.time()
        answers.append(genetic_backpack(w, items_w, items_c))
        timeExecuted += time.time() - start
    #answers.append(genetic_backpack(w, items_w, items_c))
    print(f"TIME: {timeExecuted/100*1000}")
    bestResult = max(list(map(lambda x: x[1], answers)))
    print(f'COST: {bestResult}')
    for i in answers:
        if i[1] == bestResult:
            print(f'ITEMS: {i[0]}')
            break

BENCHMARK p01
TIME: 184.08037185668945
COST: 309
ITEMS: [1. 1. 1. 1. 0. 1. 0. 0. 0. 0.]
BENCHMARK p02
TIME: 152.61001110076904
COST: 51
ITEMS: [0. 1. 1. 1. 0.]
BENCHMARK p03
TIME: 159.9299955368042
COST: 150
ITEMS: [1. 1. 0. 0. 1. 0.]
BENCHMARK p04
TIME: 167.260000705719
COST: 107
ITEMS: [1. 0. 0. 1. 0. 0. 0.]
BENCHMARK p05
TIME: 174.54999923706055
COST: 900
ITEMS: [1. 0. 1. 1. 1. 0. 1. 1.]
BENCHMARK p06
TIME: 166.07998609542847
COST: 1735
ITEMS: [0. 1. 0. 1. 0. 0. 1.]
BENCHMARK p07
TIME: 228.30999612808228
COST: 1436
ITEMS: [1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1.]


In [62]:
def genetic_salesman(graph):
    n_edges = len(graph)
    n_chromo = 100 # должно быть четным
    k = 2
    population = []
    best_solution = [99999999, []]
    
    def fitness_function(chromo):
        res = 0
        for i in range(n_edges-1):
            res += graph[chromo[i]][chromo[i+1]]
        return res
    
    def crossover(chromos_selected):    
        sep = random.choices([i for i in range(n_edges)], k=2)
        while len(np.unique(sep, axis=0)) < k:
            sep = random.choices([i for i in range(n_edges)], k=2)
        sep.sort()
        chromos_crossovered = [[-1 for i in range(n_edges)], [-1 for i in range(n_edges)]]
        pairs = []
        collisions = [[],[]]
        for i in range(sep[0], sep[1]+1):
            pairs.append((chromos_selected[0][i], chromos_selected[1][i]))
            chromos_crossovered[0][i] = chromos_selected[1][i]
            chromos_crossovered[1][i] = chromos_selected[0][i]
            collisions[0].append(chromos_selected[1][i])
            collisions[1].append(chromos_selected[0][i])
        for i in range(n_edges):
            if chromos_selected[0][i] not in collisions[0] and chromos_crossovered[0][i] == -1:
                chromos_crossovered[0][i] = chromos_selected[0][i]
                collisions[0].append(chromos_selected[0][i])
            if chromos_selected[1][i] not in collisions[1] and chromos_crossovered[1][i] == -1:
                chromos_crossovered[1][i] = chromos_selected[1][i]
                collisions[1].append(chromos_selected[1][i])
        for i in range(n_edges):
            if chromos_selected[0][i] in collisions[0] and chromos_crossovered[0][i] == -1:
                temp = chromos_selected[0][i]
                temp_pairs = pairs.copy()
                while temp in collisions[0]:
                    for p in temp_pairs:
                        if temp == p[0]:
                            temp = p[1]
                            temp_pairs.remove(p)
                            break
                        elif temp == p[1]:
                            temp = p[0]
                            temp_pairs.remove(p)
                            break
                chromos_crossovered[0][i] = temp

            if chromos_selected[1][i] in collisions[1] and chromos_crossovered[1][i] == -1:
                temp = chromos_selected[1][i]
                temp_pairs = pairs.copy()
                while temp in collisions[1]:
                    for p in temp_pairs:
                        if temp == p[0]:
                            temp_pairs.remove(p)
                            temp = p[1]
                            break
                        elif temp == p[1]:
                            temp = p[0]
                            temp_pairs.remove(p)
                            break
                chromos_crossovered[1][i] = temp
        return chromos_crossovered
    
    def mutation(chromos_crossovered):
        for c in chromos_crossovered:
            if random.randint(0, 100) < 20:
                i, j = random.randint(0, n_edges-1), random.randint(0, n_edges-1)
                c[i], c[j] = c[j], c[i]
        return chromos_crossovered
    
    def initial_solution():
        for i in range(n_chromo):
            population.append([i for i in range(n_edges)])
            random.shuffle(population[i])
        return population
    
    for i in range(100):
        if i == 0:
            population = initial_solution()
        fitness_values = []
        fitness_sum = 0
        for i in range(n_chromo):
            ff = fitness_function(population[i])
            fitness_values.append((i, ff))
            fitness_sum += ff
        fitness_values.sort(key=lambda x: x[1])
        if fitness_values[0][1] <= best_solution[0]:
            best_solution = [fitness_values[0][1], population[fitness_values[0][0]]]
        
        distribution = [0 for i in range(n_chromo)]
        for x in fitness_values:
            distribution[x[0]] = x[1] / fitness_sum

        best_chromos = random.choices(population, weights=distribution, k=k)
        while len(np.unique(best_chromos, axis=0)) < k:
            best_chromos = random.choices(population, weights=distribution, k=k)

        new_population = []
        for i in range(n_chromo // 2 - 1):
            chromos_selected = random.sample(population, 2)
            population.remove(chromos_selected[0])
            population.remove(chromos_selected[1])
            chromos_crossovered = crossover(chromos_selected)
            chromos_mutated = mutation(chromos_crossovered)
            new_population.append(chromos_mutated[0])
            new_population.append(chromos_mutated[1])

        new_population.append(best_chromos[0])
        new_population.append(best_chromos[1])

        population = new_population.copy()

    return best_solution

In [63]:
m = np.array([   [8 , 8 , 4 , 3 , 9 , 8 , 8 , 5 ,12 , 3],
                 [8 , 1 , 7 , 9 ,12 , 1 , 7 ,10 , 1 ,14],
                 [4 , 7 , 7 ,14 , 5 ,15 ,14 ,10 , 7 ,14],
                 [3 , 9 ,14 ,15 , 5 , 8 ,11 ,10 , 2 , 7],
                 [9 ,12 , 5 , 5 , 4 ,14 , 3 , 6 , 1 , 8],
                 [8 , 1 ,15 , 8 ,14 , 8 , 9 , 6 , 1 ,14],
                 [8 , 7 ,14 ,11 , 3 , 9 ,13 , 7 ,14 ,10],
                 [5 ,10 ,10 ,10 , 6 , 6 , 7 , 3 , 8 ,12],
                [12 , 1 , 7 , 2 , 1 , 1 ,14 , 8 ,11 , 7],
                 [3 ,14 ,14 , 7 , 8 ,14 ,10 ,12 , 7 , 9]])
answer = genetic_salesman(m)
print(answer)

[37, [8, 1, 5, 7, 6, 4, 3, 9, 0, 2]]


In [65]:
salesman_benchmarks = {'a280': open('benchmarks_salesman/a280.tsp', 'r').readlines()[6:-1],
                           'att48': open('benchmarks_salesman/att48.tsp', 'r').readlines()[6:-1],
                           'bays29': open('benchmarks_salesman/bays29.tsp', 'r').readlines()[38:-1],
                           'ch150': open('benchmarks_salesman/ch150.tsp', 'r').readlines()[6:-1],
                           'fl417': open('benchmarks_salesman/fl417.tsp', 'r').readlines()[6:-1],
                           }

for benchmark in salesman_benchmarks:
    coord = []
    for line in salesman_benchmarks[benchmark]:
        coord.append((float(line.split()[1]), float(line.split()[2])))
    w = []
    for i in range(len(coord)):
        w.append([])
        for j in range(len(coord)):
            w[i].append(euclidean_distance(coord[i], coord[j]))
    answers = []
    timeExecuted = 0
    for i in range(1):
        start = time.time()
        answer = genetic_salesman(w)
        timeExecuted += time.time() - start
        answers.append(answer)

    print(f"BENCHMARK {benchmark}")
    print(f"TIME {timeExecuted/100*1000}")
    bestResult = min(list(map(lambda x: x[0], answers)))
    print(f'BEST RESULT: {bestResult}')
    for i in answers:
        if i[1] == bestResult:
            print(f'PATH: {i[1]}')
            break

BENCHMARK a280
TIME 175.0600028038025
BEST RESULT: 30217.56288685789
BENCHMARK att48
TIME 11.711959838867188
BEST RESULT: 106475.68611221906
BENCHMARK bays29
TIME 6.429989337921143
BEST RESULT: 19052.86152333232
BENCHMARK ch150
TIME 59.06999588012695
BEST RESULT: 46647.887046751785
BENCHMARK fl417
TIME 374.74966764450073
BEST RESULT: 442759.0985279381
